In [2]:
import ipywidgets.widgets as widgets
from IPython.display import display
image_display = widgets.Image(format="jpeg", width="45%")
full_display = widgets.Image(format="jpeg", width="45%")
display(widgets.HBox([image_display, full_display]))

location_coords_display = widgets.Label()
display(location_coords_display)

import cv2
import pyzed.sl as sl
camera = sl.Camera()
camera_params = sl.InitParameters()
camera_params.camera_resolution = sl.RESOLUTION.VGA
camera_params.depth_mode = sl.DEPTH_MODE.ULTRA
camera_params.coordinate_units = sl.UNIT.MILLIMETER

camera_status = camera.open(camera_params)
if camera_status != sl.ERROR_CODE.SUCCESS:
    print("camera error")
    print(camera_status)
    camera.close()
    exit()

# initialize robot
import motors
robot = motors.MotorsYukon()
robot.stop()

# initialize model
import tracking
m = tracking.Model()

# get initial image and choose object to track
image_mat = sl.Mat()
depth_mat = sl.Mat()
started_tracking = False
while not started_tracking:
    err = camera.grab()
    if err == sl.ERROR_CODE.SUCCESS:
        camera.retrieve_image(image_mat)
        image = image_mat.get_data()
        
        image_display.value = m.show_all_boxes(image)
    
        user_input = input("enter id to track (or leave blank to skip):")
        if user_input == "":
            continue
        else:
            m.tracked_id = int(user_input)
            print("tracking object id " + str(user_input))
            started_tracking = True

# movement parameters
KEEP_X_BETWEEN = (275, 417)
TURN_SPEED = 0.3
KEEP_DEPTH_BETWEEN = (750, 1250)
MOVE_SPEED = 0.3

# start tracking
running = True
while running:
    err = camera.grab()
    if err == sl.ERROR_CODE.SUCCESS:
        camera.retrieve_image(image_mat)
        image = image_mat.get_data()

        camera.retrieve_measure(depth_mat, measure=sl.MEASURE.DEPTH)
        depth = depth_mat.get_data()
        
        tracked_box = m.track(image, return_type="corners")
        
        if tracked_box is False:
            image_display.value = bytes(cv2.imencode('.jpg', image)[1])
            robot.stop()
        else:
            x_centre = int((tracked_box[0] + tracked_box[2]) / 2)
            y_centre = int((tracked_box[1] + tracked_box[3]) / 2)
            image_rect = cv2.rectangle(
                image,
                (int(tracked_box[0]), int(tracked_box[1])),
                (int(tracked_box[2]), int(tracked_box[3])),
                (255, 0, 0),
                4
            )
            image_display.value = bytes(cv2.imencode('.jpg', image_rect)[1])

            # turn if tracked person is outside KEEP_X_BETWEEN
            if x_centre < KEEP_X_BETWEEN[0]:
                robot.left(speed=TURN_SPEED)
                print("turning left   ", end="\r")
            elif x_centre > KEEP_X_BETWEEN[1]:
                robot.right(speed=TURN_SPEED)
                print("turning right  ", end="\r")
            else:
                # only correct distance if target is centred
                depth_value = depth[y_centre, x_centre]
                if depth_value < KEEP_DEPTH_BETWEEN[0]:
                    robot.backward(speed=MOVE_SPEED)
                    print("turning back   ", end="\r")
                elif depth_value > KEEP_DEPTH_BETWEEN[1]:
                    robot.forward(speed=MOVE_SPEED)
                    print("turning forward", end="\r")
                else:
                    robot.stop()
                    print("stopping       ", end="\r")

        full_display.value = m.show_all_boxes(image)
    

camera.close()

Label(value='')

[2025-04-30 11:47:15 UTC][ZED][INFO] Logging level INFO
[2025-04-30 11:47:15 UTC][ZED][INFO] Logging level INFO
[2025-04-30 11:47:16 UTC][ZED][INFO] Logging level INFO
[2025-04-30 11:47:16 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-04-30 11:47:17 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-04-30 11:47:17 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-04-30 11:47:17 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-04-30 11:47:17 UTC][ZED][INFO] [Init]  Serial Number: S/N 35159485
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolo11l_half.engine for TensorRT inference...
[04/30/2025-12:47:18] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[04/30/2025

enter id to track (or leave blank to skip): 
enter id to track (or leave blank to skip): 1


tracking object id 1
new box id: tensor(13.)
    coords:  tensor([324.4466,  47.2000, 439.2500, 274.6270])
last known:  tensor([344.8659,  73.6986, 419.7677, 168.4437])
[tensor(True), tensor(True), tensor(True), tensor(False)]
3

new box id: tensor(13.)
    coords:  tensor([275.3571,  34.0879, 361.5560, 284.7193])
last known:  tensor([304.7424,  14.2878, 450.5348, 305.5343])
[tensor(True), tensor(True), tensor(False), tensor(True)]
3

new box id: tensor(147.)
    coords:  tensor([350.7310, 193.9156, 382.2695, 226.7589])
last known:  tensor([361.5024, 196.2532, 379.3966, 230.0090])
[tensor(True), tensor(True), tensor(True), tensor(True)]
4

new box id: tensor(149.)
    coords:  tensor([157.1515, 180.0323, 187.3566, 232.2339])
last known:  tensor([346.6697, 193.5538, 382.3452, 226.3625])
[tensor(False), tensor(True), tensor(False), tensor(True)]
2

new box id: tensor(153.)
    coords:  tensor([132.2879, 188.0836, 164.5097, 226.4072])
last known:  tensor([346.6697, 193.5538, 382.3452, 226

KeyboardInterrupt: 